##**Mounting Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Importing Libraries**

In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.4 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

#Hyperparameter Optimization
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-8-ff41bca45932>:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


##**Determining File Path and Separating into Subfolders**

In [ ]:
data_dir = '/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/'

data_path = "/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/"
cataract_path = os.path.join(data_path, "Cataract")
diabetic_path = os.path.join(data_path, "Diabetic Retinopathy")
glaucoma_path = os.path.join(data_path, "Glaucoma")
normal_path = os.path.join(data_path, "Normal")

print(os.listdir("/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/"))

['Glaucoma', 'Normal', 'Cataract', 'Diabetic Retinopathy']


##**Specifying Image Size**

In [ ]:
img_height, img_width = 128, 128

##**Loading Images and Concatenating the Data Set**

In [ ]:
def load_images(folder_path, label):
    images = []
    labels = []
    for idx, filename in enumerate(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error: Unable to read image {img_path}")
            continue
        img = cv2.resize(img, (img_height, img_width))
        images.append(img)
        labels.append(label)
        print(f"Image {idx + 1} loaded from {img_path}")
    return np.array(images), np.array(labels)

cataract_images, cataract_labels = load_images(cataract_path, 0)
diabetic_images, diabetic_labels = load_images(diabetic_path, 1)
glaucoma_images, glaucoma_labels = load_images(glaucoma_path, 2)
normal_images, normal_labels = load_images(normal_path, 3)

x = np.concatenate((cataract_images, diabetic_images, glaucoma_images, normal_images), axis=0)
y = np.concatenate((cataract_labels, diabetic_labels, glaucoma_labels, normal_labels), axis=0)

Image 1 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/1167_right.jpg
Image 2 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/1115_left.jpg
Image 3 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2074_right.jpg
Image 4 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2163_right.jpg
Image 5 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/1468_left.jpg
Image 6 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2106_right.jpg
Image 7 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2101_left.jpg
Image 8 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/103_left.jpg
Image 9 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2151_left.jpg
Image 10 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Cataract/2126_right.jpg
Image

##**Displaying the Number of Data Samples for X and Y**

In [ ]:
print("Number of data samples:", x.shape[0])
print("Number of data samples:", y.shape[0])

Number of data samples: 4217
Number of data samples: 4217


##**Splitting the Data Set into Training (80%) and Test (20%) Sets**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 38)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 38)

##**Training and Finalizing the CNN Model**

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=4)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=4)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=4)

In [ ]:
def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv1', min_value=0.2, max_value=0.5, step=0.1)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 3
    model.add(Dense(units=hp.Int('dense3_units', min_value=64, max_value=256, step=16)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(4, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
# Creating the tuner using RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Maximum number of models to try
    directory='my_dir',  # Directory to store saved models and results
    project_name='retinal_imaging_hyperparameter_tuning'  # Name of the tuning project
)

# Fitting the tuner with training data
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# Getting the best hyperparameters and building the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Training the best model on the training set
best_model.fit(x_train, train_yCl, epochs=15, validation_data=(x_val, valid_yCl))

Trial 5 Complete [00h 02m 24s]
val_accuracy: 0.7781065106391907

Best val_accuracy So Far: 0.7781065106391907
Total elapsed time: 00h 09m 50s
Epoch 1/15
95/95 [==============================] - 11s 98ms/step - loss: 23.1390 - accuracy: 0.5387 - val_loss: 0.8019 - val_accuracy: 0.6923
Epoch 2/15
95/95 [==============================] - 9s 95ms/step - loss: 0.7967 - accuracy: 0.6524 - val_loss: 0.7088 - val_accuracy: 0.7071
Epoch 3/15
95/95 [==============================] - 9s 93ms/step - loss: 0.7266 - accuracy: 0.6774 - val_loss: 0.6434 - val_accuracy: 0.6746
Epoch 4/15
95/95 [==============================] - 9s 94ms/step - loss: 0.6691 - accuracy: 0.6932 - val_loss: 0.6390 - val_accuracy: 0.6982
Epoch 5/15
95/95 [==============================] - 9s 92ms/step - loss: 0.6294 - accuracy: 0.7094 - val_loss: 0.6210 - val_accuracy: 0.7219
Epoch 6/15
95/95 [==============================] - 9s 93ms/step - loss: 0.6199 - accuracy: 0.7308 - val_loss: 0.6008 - val_accuracy: 0.7456
Epoch 7/15

In [ ]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

11/11 [==============================] - 0s 19ms/step - loss: 0.6593 - accuracy: 0.7160
Validation Accuracy:  0.715976357460022
27/27 [==============================] - 1s 36ms/step - loss: 0.6446 - accuracy: 0.7109
Test Accuracy:  0.7109004855155945
95/95 [==============================] - 2s 16ms/step - loss: 0.5092 - accuracy: 0.7819
Training Accuracy:  0.7818781137466431


In [ ]:
# Making predictions on the test data
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Calculating metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))

# Printing the results
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Displaying the Confusion Matrix
print('Confusion Matrix:')
print(conf_matrix)

# Showing the Classification Report
print('Classification Report:')
print(class_report)

27/27 [==============================] - 1s 14ms/step
Test Accuracy: 0.7109
Precision: 0.7074
Recall: 0.7109
F1 Score: 0.6912
Confusion Matrix:
[[ 63   0  77  58]
 [  0 218   0   2]
 [ 39   0 114  54]
 [  1   1  12 205]]
Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.32      0.42       198
           1       1.00      0.99      0.99       220
           2       0.56      0.55      0.56       207
           3       0.64      0.94      0.76       219

    accuracy                           0.71       844
   macro avg       0.70      0.70      0.68       844
weighted avg       0.71      0.71      0.69       844



##**Splitting the Data Set into Training (65%) and Test (35%) Sets**

In [ ]:
# Veri setini ayırma
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.35, random_state = 38)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 38)

##**Training and Finalizing the CNN Model**

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=4)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=4)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=4)

In [ ]:
def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv1', min_value=0.2, max_value=0.5, step=0.1)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 3
    model.add(Dense(units=hp.Int('dense3_units', min_value=64, max_value=256, step=16)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(4, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
# Creating the tuner using RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Maximum number of models to try
    directory='my_dir',  # Directory to store saved models and results
    project_name='retinal_imaging_hyperparameter_tuning'  # Name of the tuning project
)

# Fitting the tuner with training data
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# Getting the best hyperparameters and building the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Training the best model on the training set
best_model.fit(x_train, train_yCl, epochs=15, validation_data=(x_val, valid_yCl))

Reloading Tuner from my_dir/pneumonia_hyperparameter_tuning/tuner0.json
Epoch 1/15
78/78 [==============================] - 11s 118ms/step - loss: 34.7357 - accuracy: 0.5061 - val_loss: 0.9760 - val_accuracy: 0.5745
Epoch 2/15
78/78 [==============================] - 7s 93ms/step - loss: 0.8362 - accuracy: 0.6285 - val_loss: 0.7841 - val_accuracy: 0.6436
Epoch 3/15
78/78 [==============================] - 7s 93ms/step - loss: 0.7251 - accuracy: 0.6711 - val_loss: 0.6998 - val_accuracy: 0.6400
Epoch 4/15
78/78 [==============================] - 7s 92ms/step - loss: 0.6912 - accuracy: 0.6817 - val_loss: 0.7133 - val_accuracy: 0.6691
Epoch 5/15
78/78 [==============================] - 7s 93ms/step - loss: 0.6547 - accuracy: 0.6975 - val_loss: 0.6717 - val_accuracy: 0.7164
Epoch 6/15
78/78 [==============================] - 7s 92ms/step - loss: 0.6556 - accuracy: 0.6938 - val_loss: 0.6539 - val_accuracy: 0.6945
Epoch 7/15
78/78 [==============================] - 7s 92ms/step - loss: 0.6363

In [ ]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

9/9 [==============================] - 0s 19ms/step - loss: 0.6557 - accuracy: 0.6727
Validation Accuracy:  0.6727272868156433
47/47 [==============================] - 1s 22ms/step - loss: 0.5605 - accuracy: 0.7385
Test Accuracy:  0.7384823560714722
78/78 [==============================] - 1s 15ms/step - loss: 0.4903 - accuracy: 0.7944
Training Accuracy:  0.7944039106369019


In [ ]:
# Making predictions on the test data
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Calculating metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))

# Printing the results
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Displaying the Confusion Matrix
print('Confusion Matrix:')
print(conf_matrix)

# Showing the Classification Report
print('Classification Report:')
print(class_report)

47/47 [==============================] - 1s 14ms/step
Test Accuracy: 0.7385
Precision: 0.7367
Recall: 0.7385
F1 Score: 0.7240
Confusion Matrix:
[[159   0 109  86]
 [  0 383   0   2]
 [ 80   0 182  99]
 [  1   0   9 366]]
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.45      0.54       354
           1       1.00      0.99      1.00       385
           2       0.61      0.50      0.55       361
           3       0.66      0.97      0.79       376

    accuracy                           0.74      1476
   macro avg       0.73      0.73      0.72      1476
weighted avg       0.74      0.74      0.72      1476

